<a href="https://colab.research.google.com/github/aissahm/AdaptiveKDS/blob/main/AdaptiveKDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **AdaptiveKDS**

In [ ]:
%%capture

import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

# mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

!  chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d meowmeowmeowmeowmeow/gtsrb-german-traffic-sign

import zipfile
with zipfile.ZipFile("/content/gtsrb-german-traffic-sign.zip","r") as zip_ref:
    zip_ref.extractall("/content/")

In [ ]:
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Input

from tensorflow import keras
import random as random
from sklearn.metrics import accuracy_score

In [ ]:
def load_data(data_dir):
  rawimages = []
  rawlabels = []
  for label in os.listdir(data_dir):
      label_path = os.path.join(data_dir, label)
      for image_file in os.listdir(label_path):
          image_path = os.path.join(label_path, image_file)
          image = cv2.imread(image_path)
          image = cv2.resize(image, (32, 32))  # Resize images to 32x32 pixels
          rawimages.append(image)
          rawlabels.append(int(label))  # Convert folder name (label) to integer
  rawimages = np.array(rawimages)
  rawlabels = np.array(rawlabels)
  return rawimages, rawlabels

In [ ]:
images,labels = load_data('/content/Train')

# Normalize the images
normalized_images = images / 255.0

In [ ]:
import pickle

#given the dataset X, Y, the object with indexes for every client, returns the dataset of client identified with its client_id
def returnClientDataset(client_id, clients_data_obj, x, y):
  dataset_indexes = np.array(clients_data_obj[client_id]["indexes"])
  return [x[dataset_indexes], y[dataset_indexes]]

##FL parameters
clients_datasets_obj_filename = "/content/German_Traffic_Sign_100_clients_alpha_75_third.pickle"
clients_datasets_obj = pickle.load( open(clients_datasets_obj_filename, "rb" ) )

In [ ]:
def plot_history(history):
  plt.figure(figsize=(5, 3))
  plt.plot(history['accuracy'], label= history['exp_name'])
  plt.xlabel('Epoch')
  plt.ylabel('Value')
  plt.title('Training History')
  plt.legend()
  plt.grid(True)
  plt.show()

def knowledge_distillation_loss(y_true, y_pred):
  y_true = tf.convert_to_tensor(y_true, dtype=tf.float32)
  # Ensure that y_pred has the same shape as soft targets
  y_pred = tf.convert_to_tensor(y_pred, dtype=tf.float32)

  loss_ce = losses.sparse_categorical_crossentropy(y_true, y_pred, from_logits=False)
  return loss_ce

# Define a simple CNN model
def create_model():
  model = Sequential([
    Input(shape=(32, 32, 3)),
    Conv2D(16, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(43, activation='softmax')
  ])
  model.compile(loss=knowledge_distillation_loss, optimizer="adam", metrics=["accuracy"])
  return model

#Given the weights after training and initial weights, returns the gradient from entire training
def computeClientGradientNoCompression(modelNotTrained, modelTrained):
  gradient = []
  notTrainedWeight = modelNotTrained.get_weights()
  i = 0
  for weight in modelTrained.get_weights():
    gradient.append( notTrainedWeight[i] - weight )
    i += 1
  return gradient

#add the client gradient to the global model
def addGradientNoCompression(modelNotTrained, gradient, clientweight):
  newWeight = []
  i = 0
  for weight in modelNotTrained.get_weights():
    newWeight.append( weight - (gradient[i] * clientweight) )
    i += 1
  modelNotTrained.set_weights(newWeight)
  return newWeight

# Federated averaging function
def federated_avg(teacher_model, models, clientweightslst):
  gradients = []
  for model in models:
    gradient = computeClientGradientNoCompression(teacher_model, model)
    gradients.append(gradient)
  for i in range(len(models)):
    addGradientNoCompression(teacher_model, gradients[i], , clientweightslst[i])
  return teacher_model.get_weights()

# Federated training function for each client
def train_client(model, data, teacher_model, lambdaval):
  (client_x_train, client_y_train) = data

  soft_target_train = teacher_model.predict(client_x_train)

  def student_knowledge_distillation_loss(y_true, y_pred):
    loss_ce = losses.sparse_categorical_crossentropy(y_true, y_pred, from_logits=False)
    loss_kd = tf.keras.losses.KLD(soft_target_train, tf.nn.softmax(y_pred / temperature))
    return (1- lambdaval)*loss_ce + lambdaval * loss_kd  # Adjust the weight for the distillation loss as needed

  # Compile model with combined loss
  model.compile(loss=student_knowledge_distillation_loss,
                optimizer='adam', metrics=['accuracy'])
  model.set_weights(teacher_model.get_weights())
  model.fit(client_x_train, client_y_train, epochs=2, validation_split = .0 ,verbose=0,  batch_size=1)
  return model

#returns a copy of the global model to client
def returnCopyGlobalModelToClient(globalmodel):
  clientmodel = create_model()
  clientmodel.set_weights(globalmodel.get_weights())
  return clientmodel

### **setting the experiment parameters**

In [ ]:
experiments_obj = []

num_rounds = 35
participating_clients_per_round = 8
clients_per_round = 4

num_clients = 90

x_train, y_train = [normalized_images, labels]

test_indexes = []
for i in range(num_clients, len(clients_datasets_obj)):
  test_indexes += list(clients_datasets_obj[i]["indexes"])
x_test, y_test = [ x_train[test_indexes], y_train[test_indexes]  ]

train_indexes = []
for i in range(0, num_clients):
  train_indexes += list(clients_datasets_obj[i]["indexes"])

x_train_aggregated, y_train_aggregated = [x_train[train_indexes], y_train[train_indexes]]

In [ ]:
##KD parameters
experiment_name = "lambda=min(accuracy, 0.5)"
temperature = 5

In [ ]:
#######training starts
initial_model = create_model()
teacher_model = returnCopyGlobalModelToClient(initial_model)


# Initialize lists to store test accuracies and losses
test_accuracies = []
test_losses = []

# Evaluate teacher model on test data initially
loss, acc = teacher_model.evaluate(x_test, y_test)
test_accuracies.append(acc)
test_losses.append(loss)
print(f"Round:{0}, Test Loss: {loss}, Test Accuracy: {acc}")

#evaluate teacher model on local data initially
loss, acc = teacher_model.evaluate(x_train_aggregated, y_train_aggregated)
average_local_accuracy_list.append(acc)
print(f"Round:{0}, Local Loss: {loss}, Local Accuracy: {acc}")


for round in range(0, num_rounds):
  # Select participating clients for this round
  random_selected_clients = random.sample(range(num_clients), participating_clients_per_round)

  #select clients for training
  clients_accuracy_list = []
  for i, client_id in enumerate(random_selected_clients):
    client_data_i = returnClientDataset(client_id, clients_datasets_obj, x_train, y_train)
    loss, acc = teacher_model.evaluate(client_data_i[0], client_data_i[1])
    clients_accuracy_list.append({"clientID": client_id, "accuracy": acc})

  # POC strategy
  #order the accuracies from worst to best
  clients_accuracy_list.sort(key=lambda x: x["accuracy"], reverse=False)

  print("clients_accuracy_list: ", clients_accuracy_list)

  selected_clients = [item['clientID'] for item in clients_accuracy_list[:clients_per_round]]

  # Create client models (one per selected client)
  client_models = [returnCopyGlobalModelToClient(teacher_model) for _ in selected_clients]

  lambda_values = []
  clients_weights_list = []
  for i, client_id in enumerate(selected_clients):
    lambda_value = min(clients_accuracy_list[i]['accuracy'], 0.5)
    lambda_values.append(lambda_value)
    client_data_i = returnClientDataset(client_id, clients_datasets_obj, x_train, y_train)
    client_models[i] = train_client(client_models[i], client_data_i, teacher_model, lambda_value)
    clients_weights_list.append( client_data_i[1].shape[0] / y_train.shape[0] )

  print(lambda_values)

  # Update teacher model with FedAvg
  teacher_model.set_weights(federated_avg(teacher_model, client_models, clients_weights_list))

  # Evaluate teacher model on test data
  loss, acc = teacher_model.evaluate(x_test, y_test)
  test_accuracies.append(acc)
  test_losses.append(loss)
  print(f"Round:{round+1}, Test Loss: {loss}, Test Accuracy: {acc}")

  print()

# evaluate teacher model on local data
clients_accuracy_list = []
for i, client_id in enumerate(range(0, num_clients)):
  client_data_i = returnClientDataset(client_id, clients_datasets_obj, x_train, y_train)
  loss, acc = teacher_model.evaluate(client_data_i[0], client_data_i[1])
  clients_accuracy_list.append({"clientID": client_id, "accuracy": acc})

# saving the training results
experiments_obj.append({"exp_name": experiment_name, "accuracy": test_accuracies, "loss": test_losses, "clients_accuracy_list": clients_accuracy_list})
plot_history(experiments_obj[-1])